<a href="https://colab.research.google.com/github/egorostap/other_projects/blob/main/fb_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install facebookads==2.5.0
from facebookads.api import FacebookAdsApi
!pip install facebook_business
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adreportrun import AdReportRun
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.adobjects.campaign import Campaign
from facebook_business.adobjects.adset import AdSet
from facebook_business.adobjects.adaccountuser import AdAccountUser as AdUser
from facebook_business import adobjects
from matplotlib import pyplot as plt
from pandas import DataFrame
import time
from facebook_business.api import FacebookAdsApi

Написание скрипта
Создадим три переменных, в которые запишем access token, App ID и App Secret. Авторизуемся через метод init() класса FacebooksAdsApi и добавляем пользователя. Метод get_ad_accounts() вернёт нам данные по всем нашим рекламным аккаунтам в виде словаря. По этим же данным можем получить информацию о кампаниях методом get_campaigns().

In [24]:
my_access_token = 'EAAwDjsTCk8kBAMzEHZBwKO4SYud1ZBFzZAQssDUNUOjiS2kkrZBm8pZAKJHQhqfTaunLwBdUdGEVMWFEtMcflOumCKzBnZAINUQj9Iai5LoUXlrpHSaLwojz4UVD58FNyRtkDOvXkobr8VWhd7en37Ed9ugF49LxWyi4pvcjhZC7OMZC21tYMJv9rnLuP8Ix1eEVZAzicq9dnBgZDZD'
my_app_id = '3381611441853385'
my_app_secret = 'a0d38d2d8eba2fa13ff2d7246f344dd1'
FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)

# me = AdUser(fbid='me')
# my_accounts = list(me.get_ad_accounts())
# my_accounts

my_account = AdAccount('act_426823688657953')
my_accounts = [my_account]
campaigns = my_account.get_campaigns()
print(campaigns)

# my_account = my_accounts[0]


# act_426823688657953/insights?fields=campaign_name,adset_name,impressions,spend,actions&date_preset=last_30d&time_increment=1&limit=2000&level=adset

[<Campaign> {
    "id": "23848462278140216"
}, <Campaign> {
    "id": "23848205134100216"
}, <Campaign> {
    "id": "23848055655140216"
}, <Campaign> {
    "id": "23848016151470216"
}, <Campaign> {
    "id": "23847758574230216"
}, <Campaign> {
    "id": "23847551237850216"
}, <Campaign> {
    "id": "23847307645240216"
}, <Campaign> {
    "id": "23847116394610216"
}, <Campaign> {
    "id": "23847091334310216"
}, <Campaign> {
    "id": "23846900635030216"
}, <Campaign> {
    "id": "23846900473100216"
}, <Campaign> {
    "id": "23846888038520216"
}, <Campaign> {
    "id": "23846885675190216"
}]


Попробуем получить amount spent через my_account. Для этого воспользуемся методом api_get(), передав в параметр fields поле AdAccount.Field.amount_spent. Теперь, чтобы получить желаемые данные, выведем поле у переменной my_account, поделив на 100, чтобы обрубить копейки. Расходы получаем в валюте аккаунта, в нашем случае это RUB. То, ради чего мы всё это затеваем — получить данные о расходах на рекламные кампании для последующего анализа.

In [ ]:
my_account.api_get(fields=[AdAccount.Field.amount_spent])
print(int(my_account[AdAccount.Field.amount_spent])/100)

Объявим переменную fields — в этом списке будут храниться поля, которые мы хотим получать: id кампании, количество кликов, затрат и просмотров. Теперь опишем две функции. Первая будет асинхронно отправлять запросы к Facebook и возвращать результаты. Вторая — формирует эти запросы и передает в первую функцию. В результате будем получать список словарей.

In [25]:
fields = [
    AdsInsights.Field.campaign_id,
    AdsInsights.Field.clicks,
    AdsInsights.Field.spend,
    AdsInsights.Field.impressions]

count = 0

def wait_for_async_job(async_job):
    global count§
    async_job = async_job.api_get()
    while async_job[AdReportRun.Field.async_status] != 'Job Completed' or async_job[
        AdReportRun.Field.async_percent_completion] < 100:
        time.sleep(2)
        async_job = async_job.api_get()
    else:
        print("Job " + str(count) + " completed")
        count += 1
    return async_job.get_result(params={"limit": 1000})

def get_insights(account, date_preset='last_3d'):
    account = AdAccount(account["id"])
    i_async_job = account.get_insights(
        params={
            'level': 'ad',
            'date_preset': date_preset,
            'time_increment': 1},
            fields=fields,
            is_async=True)
    results = [dict(item) for item in wait_for_async_job(i_async_job)]
    return results

Следующий шаг — получение искомых данных о затратах. Будем собирать данные за всё время, поэтому заведём переменную date_preset, значение которой поставим lifetime. И для каждого аккаунта вызовем функцию get_insights(), а список, который она возвращает, положим в insights_lists.
Создадим DataFrame и вытащим из insights_lists интересующие данные — это id кампании, количество кликов, затраты и просмотры.

In [30]:


elem_insights = []
insights_lists = []
date_preset = 'last_7d'
for elem in my_accounts:
            elem_insights = get_insights(elem, date_preset)
            insights_lists.append(elem_insights)

insight_campaign_id_list = []
insight_clicks_list = []
insight_spend_list = []
insight_impressions_list = []
insight_date_start_list = []
insight_date_stop_list = []
for elem1 in insights_lists:
    for elem2 in elem1:
        insight_campaign_id_list.append(int(elem2['campaign_id']))
        insight_clicks_list.append(int(elem2['clicks']))
        insight_spend_list.append(float(elem2['spend']))
        insight_impressions_list.append(int(elem2['impressions']))
        insight_date_start_list.append(elem2['date_start'])
        insight_date_stop_list.append(elem2['date_stop'])

df = DataFrame()
df['campaign_id'] = insight_campaign_id_list
df['clicks'] = insight_clicks_list
df['spend'] = insight_spend_list
df['impressions'] = insight_impressions_list
df['date_start'] = insight_date_start_list
df['date_stop'] = insight_date_stop_list
df.head(10)

Job 4 completed


,campaign_id,clicks,spend,impressions,date_start,date_stop
0,23846885675190216,20,127.19,724,2021-11-02,2021-11-02
1,23846885675190216,41,232.39,1364,2021-11-02,2021-11-02
2,23846900473100216,1,30.10,63,2021-11-02,2021-11-02
3,23846900473100216,10,110.25,469,2021-11-02,2021-11-02
4,23846900473100216,31,219.49,834,2021-11-02,2021-11-02
5,23846900473100216,11,120.27,477,2021-11-02,2021-11-02
6,23846885675190216,30,183.36,988,2021-11-02,2021-11-02
7,23846900473100216,12,122.35,459,2021-11-02,2021-11-02
8,23846885675190216,30,229.66,997,2021-11-03,2021-11-03
9,23846900473100216,53,294.65,1520,2021-11-03,2021-11-03
